# Article #1

https://medium.com/coinmonks/building-a-simple-blockchain-data-structure-with-python-e7ebd448647a

In [1]:
import hashlib
hashlib.sha256(b"hello world").hexdigest()

'b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9'

Hashing is a process which you turn anything (as long as you can represent it as a string) into a fixed 256 bit string. In the previous example, the string “hello world” has a length 11. Actually the length of “hello world” is depended on how you count it. But for simplicity, we just count how many characters. That “hello world” string is turned into a fixed-size string, which is ‘b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9’. Say I hash another string with different length.

In [10]:
hash_str = hashlib.sha256(b"I am the best president. Ever.").hexdigest()

In [11]:
type(hash_str)

str

In [13]:
bin(0xb94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9)

'0b1011100101001101001001111011100110010011010011010011111000001000101001010010111001010010110101111101101001111101101010111111101011000100100001001110111111100011011110100101001110000000111011101001000010001000111101111010110011100010111011111100110111101001'

In [15]:
print(hashlib.sha256(b"HI!").hexdigest())
print(hashlib.sha256(b"Hi!").hexdigest())

b60eaf09e14454fe5135c8fd9aeca44a49115f5c9ed8d54db7add6ce58e6d529
ca51ce1fb15acc6d69b8a5700256172fcc507e02073e6f19592e341bd6508ab8


To describe the hashing algorithm, it is quite a work. But someday I’ll explain the code behind SHA-256. Now that you can comprehend the vastness of hashing, let’s move on. Blockchain is like a linked list, a data structure known by many computer science students. Let’s create a block. The first block in Bitcoin is called genesis block.

In [20]:
import hashlib, json

def build_block(block):
    block_serialized = json.dumps(block,
                                  sort_keys=True).encode('utf-8')
    block_hash = hashlib.sha256(block_serialized).hexdigest()
    return block_hash

block_genesis = {
    'prev_hash' : None,
    'transactions' : [1,3,5,2]
}

In [21]:
block_genesis_serialized = json.dumps(block_genesis,
                                      sort_keys=True).encode('utf-8')
block_genesis_hash = hashlib.sha256(block_genesis_serialized).hexdigest()

block_genesis_hash = build_block(block_genesis)

In [19]:
block_2 = {
    'prev_hash' : block_genesis_hash,
    'transactions' : [3,3,3,8,7, 12]
}

In [22]:
block_2_hash = build_block(block_2)

In [23]:
block_3 = {
    'prev_hash' : block_2_hash,
    'transactions' : [3,4,4,8,34]
}

In [24]:
block_3_hash = build_block(block_3)

In [29]:
block_genesis = {
 'prev_hash': None,
 'transactions': [1, 3, 4, 2]
}

block_2 = {
 'prev_hash': None,
 'transactions': [3, 3, 3, 8, 7, 12]
}

block_3 = {
 'prev_hash': None,
 'transactions': [3, 4, 4, 8, 34]
}

def hash_blocks(blocks):
    prev_hash = None
    
    for block in blocks:
        
        block['prev_hash'] = prev_hash
        block_serialized = json.dumps(block,
                                      sort_keys=True).encode('utf-8')
        
        block_hash = hashlib.sha256(block_serialized).hexdigest()
        prev_hash = block_hash
        
    return prev_hash

print("Original hash")
print(hash_blocks([block_genesis, block_2, block_3]))

print("\nTempering the data")
block_genesis['transactions'][0] = 3
print('block_genesis[\'transactions\'][0] = 3')

print("\nAfter being tampered")
print(hash_blocks([block_genesis, block_2, block_3]))

Original hash
45eda4f7a76bf0f92a0acda2ce4752dfbe167473376f766f22d7ec68501cac40

Tempering the data
block_genesis['transactions'][0] = 3

After being tampered
27d68dae05428be6aa244869196a481f431fca6645dd33c3df7a740afa03b7d9


# Article #2

https://towardsdatascience.com/building-a-minimal-blockchain-in-python-4f2e9934101d

In [1]:
import copy     # fork a chain
import datetime # get real time for timestamps
import hashlib  # hash

## 1. Define classes

In [2]:
class MinimalChain():
    def __init__(self): # initialize when creating a chain
        self.blocks = [self.get_genesis_block()]
        
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
        
    def get_genesis_block(self):
        return MinimalBlock(0,
                            datetime.datetime.utcnow(),
                            'Genesis',
                            'arbitrary')
    
    def add_block(self, data):
        self.blocks.append(MinimalBlock(len(self.blocks),
                                        datetime.datetime.utcnow(),
                                        data,
                                        self.blocks[len(self.blocks) - 1].hash))
        
    def get_chain_size(self): # exclude genesis block
        return len(self.blocks) - 1
    
    def verify(self, verbose=True):
        flag = True
        for i in range(1, len(self.blocks)):
            if not self.blocks[i].verify(): # assume Genesis block integrity
                flag = False
                if verbose:
                    print(f'Wring data type(s) at block {i}.')
            if self.blocks[i].index != i:
                flag = False
                if verbose:
                    print(f'Wrong block index at block {i}')
            if self.blocks[i-1].hash != self.blocks[i].previous_hash:
                flag = False
                if verbose:
                    print(f'Wrong previous hash at block {i}')
            if self.blocks[i-1].timestamp >= self.blocks[i].timestamp:
                flag = False
                if verbose:
                    print(f'Backdating at block {i}')
        return flag
    
    def fork(self, head='latest'):
        if head in ['latest', 'whole', 'all']:
            return copy.deepcopy(self) # deep copy since they are mutable
        else:
            c = copy.deepcopy(self)
            c.blocks = c.blocks[0:head + 1]
            return c
        
    def get_root(self, chain_2):
        min_chain_size = min(self.get_chain_size(), chain_2.get_chain_size())
        for i in range(1, min_chain_size + 1):
            if self.blocks[i] != chain_2.blocks[i]:
                return self.fork(i - 1)
        return self.fork(min_chain_size)
    

In [5]:
class MinimalBlock():
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index 
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hashing()
        
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
        
    def hashing(self):
        key = hashlib.sha256()
        key.update(str(self.index).encode('utf-8'))
        key.update(str(self.timestamp).encode('utf-8'))
        key.update(str(self.data).encode('utf-8'))
        key.update(str(self.previous_hash).encode('utf-8'))
        return key.hexdigest()
    
    def verify(self):
        instances = [self.index, self.timestamp, self.previous_hash, self.hash]
        types = [int, datetime.datetime, str, str]
        if sum(map(lambda inst_, type_: isinstance(inst_, type_), instances, types)) == len(instances):
            return True
        else:
            return False

## 2. Testing

In [6]:
c = MinimalChain()
for i in range(1, 20 + 1):
    c.add_block(f'This is block {i} of my first chain.')

In [7]:
print(c.blocks[3].timestamp)
print(c.blocks[7].data)
print(c.blocks[9].hash)

2019-09-07 17:16:29.211477
This is block 7 of my first chain.
8369056b9f1f008e31d91062739f2d748cbdc691445bb631a24a5493102ff1f4


In [8]:
print(c.get_chain_size())
print(c.verify())

20
True


In [9]:
c_forked = c.fork('latest')
print(c == c_forked)

True


In [10]:
c_forked.add_block('New block for forked chain!')
print(c.get_chain_size(), c_forked.get_chain_size())

20 21


## 3. Conflict Testing

In [11]:
c_forked = c.fork('latest')
c_forked.blocks[9].index = -9
c_forked.verify()

Wrong block index at block 9


False

In [12]:
c_forked = c.fork('latest')
c_forked.blocks[16].timestamp = datetime.datetime(2000, 1, 1, 0, 0, 0, 0)
c_forked.verify()

Backdating at block 16


False

In [13]:
c_forked = c.fork('latest')
c_forked.blocks[5].previous_hash = c_forked.blocks[1].hash
c_forked.verify()

Wrong previous hash at block 5


False

In [14]:
c_forked = c.fork('latest')
c_forked.blocks[5].previous_hash = c.blocks[5].hash
c_forked.verify()

Wrong previous hash at block 5


False